# Most common word occurece in text using TF-IDF

Text Source

* Portrait: https://www.gutenberg.org/files/4217/4217-0.txt
* Ulysses: https://www.gutenberg.org/files/4300/4300-0.txt

In [1]:
%%spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2060,application_1572292571909_0542,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


Let's get a little info about our session

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2047,application_1572292571909_0529,pyspark,idle,Link,Link,
2050,application_1572292571909_0532,pyspark,idle,Link,Link,
2051,application_1572292571909_0533,pyspark,idle,Link,Link,
2053,application_1572292571909_0535,pyspark,idle,Link,Link,
2054,application_1572292571909_0536,pyspark,idle,Link,Link,
2055,application_1572292571909_0537,pyspark,idle,Link,Link,
2056,application_1572292571909_0538,pyspark,idle,Link,Link,
2057,application_1572292571909_0539,pyspark,idle,Link,Link,
2059,application_1572292571909_0541,pyspark,idle,Link,Link,
2060,application_1572292571909_0542,pyspark,idle,Link,Link,✔


Learn a little more about sparkmagic commands

In [3]:
%%help

Magic,Example,Explanation
info,%%info,Outputs session information for the current Livy endpoint.
cleanup,%%cleanup -f,"Deletes all sessions for the current Livy endpoint, including this notebook's session. The force flag is mandatory."
delete,%%delete -f -s 0,Deletes a session by number for the current Livy endpoint. Cannot delete this kernel's session.
logs,%%logs,Outputs the current session's Livy logs.
configure,"%%configure -f{""executorMemory"": ""1000M"", ""executorCores"": 4}",Configure the session creation parameters. The force flag is mandatory if a session has already been created and the session will be dropped and recreated.Look at Livy's POST /sessions Request Body for a list of valid parameters. Parameters must be passed in as a JSON string.
spark,%%spark -o dfdf = spark.read.parquet('...,"Executes spark commands. Parameters: -o VAR_NAME: The Spark dataframe of name VAR_NAME will be available in the %%local Python context as a Pandas dataframe with the same name. -m METHOD: Sample method, either take or sample. -n MAXROWS: The maximum number of rows of a dataframe that will be pulled from Livy to Jupyter. If this number is negative, then the number of rows will be unlimited. -r FRACTION: Fraction used for sampling."
sql,%%sql -o tables -qSHOW TABLES,"Executes a SQL query against the variable sqlContext (Spark v1.x) or spark (Spark v2.x). Parameters: -o VAR_NAME: The result of the SQL query will be available in the %%local Python context as a Pandas dataframe. -q: The magic will return None instead of the dataframe (no visualization). -m, -n, -r are the same as the %%spark parameters above."
local,%%locala = 1,All the code in subsequent lines will be executed locally. Code must be valid Python code.


In [3]:
import os
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'

In [4]:
import pyspark.sql.functions as f

StopWords - to remove from from analysis

In [5]:
StopWords = ["a", "about", "above", "after", "again", "against", "ain", "all", "am", "an", "and", "any", "are", "aren", "aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "couldn", "couldn't", "d", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", "doing", "don", "don't", "down", "during", "each", "few", "for", "from", "further", "had", "hadn", "hadn't", "has", "hasn", "hasn't", "have", "haven", "haven't", "having", "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "ll", "m", "ma", "me", "mightn", "mightn't", "more", "most", "mustn", "mustn't", "my", "myself", "needn", "needn't", "no", "nor", "not", "now", "o", "of", "off", "on", "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "re", "s", "same", "shan", "shan't", "she", "she's", "should", "should've", "shouldn", "shouldn't", "so", "some", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", "through", "to", "too", "under", "until", "up", "ve", "very", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "when", "where", "which", "while", "who", "whom", "why", "will", "with", "won", "won't", "wouldn", "wouldn't", "y", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves", "could", "he'd", "he'll", "he's", "here's", "how's", "i'd", "i'll", "i'm", "i've", "let's", "ought", "she'd", "she'll", "that's", "there's", "they'd", "they'll", "they're", "they've", "we'd", "we'll", "we're", "we've", "what's", "when's", "where's", "who's", "why's", "would"]

In [6]:
import subprocess

cmd = 'hdfs dfs -ls /data/texts'.split() 
files = subprocess.check_output(cmd).strip().split('\n')
for path in files:
    print (path)

Found 2 items
-rw-r--r--   3 hdfs hdfs     496309 2018-10-11 11:04 /data/texts/Portrait.txt
-rw-r--r--   3 hdfs hdfs    1580890 2018-10-11 11:04 /data/texts/Ulysses.txt

### Ingest data

In [24]:
raw_lines = spark.read.text('/data/texts')\
    .withColumn('file', f.input_file_name())
raw_lines.select('file').show(10, truncate = False)

+---------------------------------------------------------+
|file                                                     |
+---------------------------------------------------------+
|hdfs://rapid-777.oit.duke.edu:8020/data/texts/Ulysses.txt|
|hdfs://rapid-777.oit.duke.edu:8020/data/texts/Ulysses.txt|
|hdfs://rapid-777.oit.duke.edu:8020/data/texts/Ulysses.txt|
|hdfs://rapid-777.oit.duke.edu:8020/data/texts/Ulysses.txt|
|hdfs://rapid-777.oit.duke.edu:8020/data/texts/Ulysses.txt|
|hdfs://rapid-777.oit.duke.edu:8020/data/texts/Ulysses.txt|
|hdfs://rapid-777.oit.duke.edu:8020/data/texts/Ulysses.txt|
|hdfs://rapid-777.oit.duke.edu:8020/data/texts/Ulysses.txt|
|hdfs://rapid-777.oit.duke.edu:8020/data/texts/Ulysses.txt|
|hdfs://rapid-777.oit.duke.edu:8020/data/texts/Ulysses.txt|
+---------------------------------------------------------+
only showing top 10 rows

In [9]:
lines_by_book = raw_lines \
    .withColumn('book', f.regexp_extract('file', '^.*/(.*)\.txt$', 1))\
    .drop('file')
lines_by_book.show()

+--------------------+-------+
|               value|   book|
+--------------------+-------+
|                    |Ulysses|
|The Project Guten...|Ulysses|
|                    |Ulysses|
|This eBook is for...|Ulysses|
|no restrictions w...|Ulysses|
|it under the term...|Ulysses|
|eBook or online a...|Ulysses|
|                    |Ulysses|
|                    |Ulysses|
|      Title: Ulysses|Ulysses|
|                    |Ulysses|
| Author: James Joyce|Ulysses|
|                    |Ulysses|
|Release Date: Aug...|Ulysses|
|Last Updated: Aug...|Ulysses|
|                    |Ulysses|
|   Language: English|Ulysses|
|                    |Ulysses|
|Character set enc...|Ulysses|
|                    |Ulysses|
+--------------------+-------+
only showing top 20 rows

In [10]:
# Counting the total number of words in each book
lines_by_book.withColumn('word', f.explode(f.split(f.col('value'), ' ')))\
    .groupBy('book')\
    .count()\
    .sort('count', ascending=False)\
    .show()

+--------+------+
|    book| count|
+--------+------+
| Ulysses|278497|
|Portrait| 90866|
+--------+------+

In [19]:
words_by_book = lines_by_book.withColumn('word', f.explode(f.split(f.lower(f.col('value')), '[^a-z]'))).drop('value')\
                            .where(f.length('word')>0)\
                            .filter(~f.col('word').isin(StopWords))

In [20]:
# Showing top words in each text
words_by_book.show()

+-------+------------+
|   book|        word|
+-------+------------+
|Ulysses|     project|
|Ulysses|   gutenberg|
|Ulysses|       ebook|
|Ulysses|     ulysses|
|Ulysses|       james|
|Ulysses|       joyce|
|Ulysses|       ebook|
|Ulysses|         use|
|Ulysses|      anyone|
|Ulysses|    anywhere|
|Ulysses|        cost|
|Ulysses|      almost|
|Ulysses|restrictions|
|Ulysses|  whatsoever|
|Ulysses|         may|
|Ulysses|        copy|
|Ulysses|        give|
|Ulysses|        away|
|Ulysses|         use|
|Ulysses|       terms|
+-------+------------+
only showing top 20 rows

In [22]:
# Word count in both the texts
words_by_book.groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .show()

+-------+-----+
|   word|count|
+-------+-----+
|   said| 1823|
|stephen| 1004|
|  bloom| 1001|
|    one|  965|
|   like|  951|
|     mr|  855|
|    old|  603|
|    man|  563|
|    see|  521|
|   eyes|  502|
|   time|  498|
|   says|  494|
| father|  480|
|   back|  479|
|    god|  470|
|    two|  470|
|    yes|  434|
|   know|  432|
| little|  420|
|   good|  412|
+-------+-----+
only showing top 20 rows

### Computing Term Frequency (TF) for each book-word combination.  
  
Term Frequency (TF) = word count measure normalized by document. If we let  
    $n_w$ = the  number of occurrences of a particular word in a document and  
    $n_d$ = the total number of words in a document  
  
  
$$TF = n_w / n_d$$  

In [29]:
from pyspark.sql.window import Window

w = Window.partitionBy(words_by_book['book'])

book_tf = words_by_book.groupBy('book', 'word')\
    .agg(f.count('*').alias('n_w'),
         f.sum(f.count('*')).over(w).alias('n_d'),
         (f.count('*')/f.sum(f.count('*')).over(w)).alias('tf')
        )\
    .orderBy('n_w', ascending=False)\
    .cache()

book_tf.show()

+--------+-------+----+------+--------------------+
|    book|   word| n_w|   n_d|                  tf|
+--------+-------+----+------+--------------------+
| Ulysses|   said|1208|145565|0.008298698176072546|
| Ulysses|  bloom|1000|145565|  0.0068697832583382|
| Ulysses|    one| 742|145565|0.005097379177686944|
| Ulysses|   like| 731|145565|0.005021811561845224|
| Ulysses|     mr| 719|145565|0.004939374162745165|
|Portrait|   said| 615| 43044|0.014287705603568441|
| Ulysses|stephen| 571|145565|0.003922646240511112|
| Ulysses|    old| 492|145565|0.003379933363102394|
| Ulysses|   says| 473|145565|0.003249407481193...|
| Ulysses|    man| 451|145565|0.003098272249510528|
| Ulysses|    see| 435|145565|0.002988355717377...|
|Portrait|stephen| 433| 43044|0.010059474026577456|
| Ulysses|    two| 391|145565|0.002686085254010236|
| Ulysses|   time| 380|145565|0.002610517638168516|
| Ulysses|   back| 362|145565|0.002486861539518428|
| Ulysses|    yes| 359|145565|0.002466252189743...|
| Ulysses|  

## Computing the Inverse Documents Frequency (IDF) for each book-word combination.
    $c_d$ = the total number of documents   
    $i_d$ = the number of documents with a given term in them  

then we define IDF as  
  
$$IDF = log(c_d / i_d)$$
     
wheren $log$ is a natural logarithm with base e.  

In [45]:
#w_idf = Window.partitionBy(words_by_book['book'])
from pyspark.sql.window import Window

w_d = Window.partitionBy(words_by_book['word'])

c_d = words_by_book.select('book').distinct().count()

book_idf = words_by_book.groupBy('book', 'word')\
    .agg(f.lit(c_d).alias('c_d'),
         f.count('*').over(w_d).alias('i_d'),
         (f.log(f.lit(c_d)/f.count('*').over(w_d))).alias('idf')
        )\
    .cache()

In [46]:
book_idf.show()

+--------+-------------+---+---+------------------+
|    book|         word|c_d|i_d|               idf|
+--------+-------------+---+---+------------------+
| Ulysses| accumulation|  2|  1|0.6931471805599453|
|Portrait|apprehensions|  2|  1|0.6931471805599453|
| Ulysses|       arator|  2|  1|0.6931471805599453|
| Ulysses|      arclamp|  2|  1|0.6931471805599453|
| Ulysses|          art|  2|  2|               0.0|
|Portrait|          art|  2|  2|               0.0|
| Ulysses|        baaaa|  2|  1|0.6931471805599453|
| Ulysses|     baddybad|  2|  1|0.6931471805599453|
| Ulysses|       bazaar|  2|  1|0.6931471805599453|
| Ulysses|       biting|  2|  1|0.6931471805599453|
|Portrait|     blackish|  2|  1|0.6931471805599453|
| Ulysses|  bloohimwhom|  2|  1|0.6931471805599453|
| Ulysses|      blossom|  2|  1|0.6931471805599453|
| Ulysses|       brands|  2|  1|0.6931471805599453|
| Ulysses|     cactuses|  2|  1|0.6931471805599453|
| Ulysses|        carlo|  2|  1|0.6931471805599453|
| Ulysses|  